# Steelers Games Summary Data Scrape

[pro-football reference](https://www.pro-football-reference.com/) Pro-football reference includes NFL data, dating back to 1967. This data includes player statistics, all-time leaders, draft history, coaches, and much more. Statistics are updated by every week, no later than Tuesday at 6pm. Additional data can be found behind a paid subscription.

*this overview comes from [Ohio State's Sports and Society Initiative](https://sportsandsociety.osu.edu/sports-data-sets)*

In [22]:
# packages
import pandas as pd
import numpy as np
import warnings

# scraping
import requests
from bs4 import BeautifulSoup
import re
import time
import lxml # used for parsing html

# bigquery
import os
from dotenv import load_dotenv
from google.cloud import bigquery
from datetime import datetime
from google.cloud import bigquery
import db_dtypes

### Web Scraping

In [2]:
# Base URL for the Steelers' main page
base_url = "https://www.pro-football-reference.com/teams/pit/"
headers = {"User-Agent": "Mozilla/5.0"}

# Step 1: Scrape the main page to get links for each year
response = requests.get(base_url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# check for successful request
if response.status_code != 200:
    print("Failed to retrieve the page")
    exit()

If no Failure message we are good to continue

In [3]:
# Collect all year links from the main team page
year_links_start = [
    base_url + a['href']
    for a in soup.select('table#team_index a[href]')
    if re.match(r'/teams/pit/\d{4}\.htm', a['href'])
]

In [4]:
# function to remove duplicates and preserve the order

def removeduplicate(data):
    countdict = {}
    for element in data:
        if element in countdict.keys():
            
            # increasing the count if the key(or element)
            # is already in the dictionary
            countdict[element] += 1
        else:
            # inserting the element as key  with count = 1
            countdict[element] = 1
    data.clear()
    for key in countdict.keys():
        data.append(key)
        
removeduplicate(year_links_start)

# remove //teams/pit from the urls
year_links = [link.replace("//teams/pit", "") for link in year_links_start]

year_links[:5]

['https://www.pro-football-reference.com/teams/pit/2024.htm',
 'https://www.pro-football-reference.com/teams/pit/2023.htm',
 'https://www.pro-football-reference.com/teams/pit/2022.htm',
 'https://www.pro-football-reference.com/teams/pit/2021.htm',
 'https://www.pro-football-reference.com/teams/pit/2020.htm']

In [ ]:
warnings.filterwarnings('ignore')

# loop through each year

all_games = []

for link in year_links:
    year = re.search(r'\d{4}', link).group()  # Extract the year from the link
    print(f"Scraping data for year {year}...")
    year_url = f"{link}"

    # request the year's page
    year_response = requests.get(year_url, headers=headers)
    year_soup = BeautifulSoup(year_response.text, "html.parser")

    # find the "Schedule & Game Results" table
    table = year_soup.find("table", id="games")
    if table:
        # load table into a DataFrame
        games_df = pd.read_html(str(table))[0]

        # add a column for the year
        games_df["Year"] = year

        # append to the list
        all_games.append(games_df)

    # avoid overloading the server
    time.sleep(1)
    

In [37]:
# step 3: Concatenate all dataframes into a single DataFrame
schedule_df = pd.concat(all_games, ignore_index=True)

# make a copy in case we want to go back
schedule_df2 = schedule_df.copy()

# assuming df is your DataFrame after reading the table
# flatten the multi-level column index
schedule_df2.columns = ['_'.join(col).strip() for col in schedule_df2.columns.values]

# rename columns containing 'Unnamed' to add 'Misc_' prefix and clean the rest
schedule_df2.columns = [
    'Misc_' + re.sub(r'^Unnamed:.*?_level_0_', '', col) if 'Unnamed' in col else col
    for col in schedule_df2.columns
]

# rename cols without headers on website
schedule_df2.rename(columns={'Misc_Unnamed: 3_level_1': 'Misc_Time', 'Misc_Unnamed: 4_level_1': 'Misc_Boxscore', 'Misc_Unnamed: 8_level_1': 'Misc_Location',
                             'Misc_Unnamed: 5_level_1': 'Misc_Outcome', 'Expected Points_Sp. Tms': 'Expected Points_Sp_Tms', 'Year_': 'Misc_Year'}, inplace=True)

# remove boxscore and and fix 'Expected Points_Sp. Tms'

# move year to front 
first_col = schedule_df2.pop('Misc_Year')
# insert column using insert(position,column_name, first_column) function 
schedule_df2.insert(0, 'Misc_Year', first_col) 

# drop boxscore
schedule_df3 = schedule_df2.drop('Misc_Boxscore', axis=1)

# cleanup OT overtime and Home/Away

# replace '@' with 'Away' and NaN with 'Home'
schedule_df3['Misc_Location'] = schedule_df3['Misc_Location'].replace('@', 'Away').fillna('Home')

# replace 'OT' with 'Y' and NaN with 'N'
schedule_df3['Misc_OT'] = schedule_df3['Misc_OT'].replace('OT', 'Y').fillna('N')

# add underscores to column headers instead of spaces
schedule_df3.columns = [re.sub(r' ', '_', col) for col in schedule_df3.columns]

schedule_df3.head()
#schedule_df3.columns

,Misc_Year,Misc_Week,Misc_Day,Misc_Date,Misc_Time,Misc_Outcome,Misc_OT,Misc_Rec,Misc_Location,Misc_Opp,...,Offense_RushY,Offense_TO,Defense_1stD,Defense_TotYd,Defense_PassY,Defense_RushY,Defense_TO,Expected_Points_Offense,Expected_Points_Defense,Expected_Points_Sp_Tms
0,2024,1,Sun,September 8,1:00PM ET,W,N,1-0,Away,Atlanta Falcons,...,137.0,NaN,15.0,226.0,137.0,89.0,3.0,-6.25,13.70,2.05
1,2024,2,Sun,September 15,4:25PM ET,W,N,2-0,Away,Denver Broncos,...,141.0,NaN,13.0,295.0,231.0,64.0,2.0,-1.20,6.86,-1.25
2,2024,3,Sun,September 22,1:00PM ET,W,N,3-0,Home,Los Angeles Chargers,...,114.0,1.0,10.0,166.0,105.0,61.0,NaN,10.48,6.58,-4.19
3,2024,4,Sun,September 29,1:00PM ET,L,N,3-1,Away,Indianapolis Colts,...,122.0,2.0,22.0,358.0,225.0,133.0,NaN,2.80,-12.91,4.94
4,2024,5,Sun,October 6,8:20PM ET,L,N,3-2,Home,Dallas Cowboys,...,92.0,1.0,25.0,445.0,336.0,109.0,3.0,2.75,-11.22,2.98


In [ ]:
schedule_df4 = schedule_df3.copy()

# running in to an issue with 'Misc_Week' as it contains mixed data types. we will fix this by creating two columns
# one for the regular season week numbers and one for the other text. That way we can load one as a string and one as float to BigQuery

schedule_df4['Misc_Week_Text'] = schedule_df4['Misc_Week']

# first, handle the string values in Misc Week. convert to nulls

# convert non-numeric values to NaN, then change the column type to numeric
schedule_df4['Misc_Week'] = pd.to_numeric(schedule_df4['Misc_Week'], errors='coerce')

# next, handle the new column we created in order to keep the text

# function to keep only values that contain letters
def keep_text_only(val):
    if isinstance(val, str) and re.search("[a-zA-Z]", val):
        return val  # Keep values with letters
    else:
        return np.nan  # Replace other values with NaN

# apply the function to the column
schedule_df4['Misc_Week_Text'] = schedule_df4['Misc_Week_Text'].apply(keep_text_only)


In [36]:
# quick reorder to get the Week fields next to eachother
schedule_df5 = schedule_df4[['Misc_Year', 'Misc_Week', 'Misc_Week_Text', 'Misc_Day', 'Misc_Date', 'Misc_Time',
       'Misc_Outcome', 'Misc_OT', 'Misc_Rec', 'Misc_Location', 'Misc_Opp',
       'Score_Tm', 'Score_Opp', 'Offense_1stD', 'Offense_TotYd',
       'Offense_PassY', 'Offense_RushY', 'Offense_TO', 'Defense_1stD',
       'Defense_TotYd', 'Defense_PassY', 'Defense_RushY', 'Defense_TO',
       'Expected_Points_Offense', 'Expected_Points_Defense',
       'Expected_Points_Sp_Tms']]

schedule_df5.head()

,Misc_Year,Misc_Week,Misc_Week_Text,Misc_Day,Misc_Date,Misc_Time,Misc_Outcome,Misc_OT,Misc_Rec,Misc_Location,...,Offense_RushY,Offense_TO,Defense_1stD,Defense_TotYd,Defense_PassY,Defense_RushY,Defense_TO,Expected_Points_Offense,Expected_Points_Defense,Expected_Points_Sp_Tms
0,2024,1.0,NaN,Sun,September 8,1:00PM ET,W,N,1-0,Away,...,137.0,NaN,15.0,226.0,137.0,89.0,3.0,-6.25,13.70,2.05
1,2024,2.0,NaN,Sun,September 15,4:25PM ET,W,N,2-0,Away,...,141.0,NaN,13.0,295.0,231.0,64.0,2.0,-1.20,6.86,-1.25
2,2024,3.0,NaN,Sun,September 22,1:00PM ET,W,N,3-0,Home,...,114.0,1.0,10.0,166.0,105.0,61.0,NaN,10.48,6.58,-4.19
3,2024,4.0,NaN,Sun,September 29,1:00PM ET,L,N,3-1,Away,...,122.0,2.0,22.0,358.0,225.0,133.0,NaN,2.80,-12.91,4.94
4,2024,5.0,NaN,Sun,October 6,8:20PM ET,L,N,3-2,Home,...,92.0,1.0,25.0,445.0,336.0,109.0,3.0,2.75,-11.22,2.98


In [33]:
schedule_df5.dtypes

Misc_Year                   object
Misc_Week                  float64
Misc_Week_Text              object
Misc_Day                    object
Misc_Date                   object
Misc_Time                   object
Misc_Outcome                object
Misc_OT                     object
Misc_Rec                    object
Misc_Location               object
Misc_Opp                    object
Score_Tm                   float64
Score_Opp                  float64
Offense_1stD               float64
Offense_TotYd              float64
Offense_PassY              float64
Offense_RushY              float64
Offense_TO                 float64
Defense_1stD               float64
Defense_TotYd              float64
Defense_PassY              float64
Defense_RushY              float64
Defense_TO                 float64
Expected_Points_Offense    float64
Expected_Points_Defense    float64
Expected_Points_Sp_Tms     float64
dtype: object

### Load scraped data as a table to BigQuery

In [34]:
# used for both BQ read/write

# setting environmental variable directly in your code
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'bq-crudek-data.json'

# initialize the BigQuery Client
client = bigquery.Client()

# set table_id to the ID of the table to create
table_id = 'crudek-data.practice_data.steelers_games'

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("Misc_Year", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Misc_Week_Text", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Misc_Day", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Misc_Date", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Misc_Time", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Misc_Outcome", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Misc_OT", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Misc_Rec", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Misc_Location", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("Misc_Opp", bigquery.enums.SqlTypeNames.STRING),
    ],
    write_disposition="WRITE_TRUNCATE",
)

# make API request
job = client.load_table_from_dataframe(
    schedule_df5, table_id, job_config=job_config
)  
# wait for the job to complete.
job.result()  

LoadJob<project=crudek-data, location=US, id=1bbb9f92-8a30-4a37-9418-03e6a04e04f3>

In [35]:
# confirm with shape
table = client.get_table(table_id)
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 709 rows and 26 columns to crudek-data.practice_data.steelers_games
